# Importing Libraries

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
# import sys
# np.set_printoptions(threshold=sys.maxsize)

# Data 

In [2]:
data = pd.read_excel('todoData.xlsx', date_parser=True)
data = data.set_index('Date')
data.head()

,day,month,year,sum,bread,egg,waffles,butter,milk,yogurt,...,strawberry,peach,coriander,mint,tomato,onion,potatoes,garlic,ginger,pepper
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1,1,18,1118,1,1,1,1,1,1,...,0,1,1,0,1,1,1,1,1,0
2018-01-02,2,1,18,2118,1,0,0,0,1,1,...,0,0,1,0,0,0,0,0,0,0
2018-01-03,3,1,18,3118,0,0,0,0,1,1,...,0,0,0,0,1,0,0,0,0,0
2018-01-04,4,1,18,4118,1,1,0,0,1,0,...,0,0,1,1,1,0,0,0,0,0
2018-01-05,5,1,18,5118,1,1,0,0,1,1,...,1,1,1,0,1,0,0,1,1,1


In [3]:
data.describe()

,day,month,year,sum,bread,egg,waffles,butter,milk,yogurt,...,strawberry,peach,coriander,mint,tomato,onion,potatoes,garlic,ginger,pepper
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,...,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,15.720548,6.526027,18.500000,52317.678082,0.617808,0.516438,0.108219,0.321918,0.867123,0.735616,...,0.200000,0.398630,0.667123,0.249315,0.832877,0.200000,0.080822,0.250685,0.250685,0.200000
std,8.802278,3.450215,0.500343,76892.181651,0.486256,0.500072,0.310870,0.467532,0.339674,0.441307,...,0.400274,0.489952,0.471566,0.432913,0.373342,0.400274,0.272748,0.433705,0.433705,0.400274
min,1.000000,1.000000,18.000000,1118.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,4.000000,18.000000,11118.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,16.000000,7.000000,18.500000,20918.500000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23.000000,10.000000,19.000000,30918.750000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.750000,0.750000,0.000000
max,31.000000,12.000000,19.000000,311219.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Spliting Training, Test Data

In [4]:
train ,test= data.iloc[:-100,3:] , data.iloc[-100:,3:]

# Normalizing Values

In [5]:
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

C:\Users\FC\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Preparing Training Data 

In [6]:
# training on 3 days then predciting 4th value

x_train = []
y_train = []

for i in range (3, train.shape[0]):
    x_train.append(train[i-3:i])
    y_train.append(train[i])    

### scaling traing data 

In [7]:
#converting traning data into munpy array

x_train , y_train = np.array(x_train), np.array(y_train)

y_train = y_train[:,1:]

print(x_train.shape)
print(y_train.shape)


(627, 3, 30)
(627, 29)


# Build Model

In [8]:
# building LSTM network

numbers_of_features = 30

from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

regressor = Sequential()

regressor.add(LSTM(units = 140, activation = "relu", return_sequences = True, input_shape= (x_train.shape[1],30)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = "relu", return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 100, activation = "relu", return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = "relu"))
regressor.add(Dropout(0.2))
regressor.add(Dense(29, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
regressor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 3, 140)            95760     
_________________________________________________________________
dropout (Dropout)            (None, 3, 140)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 120)            125280    
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 120)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 100)            88400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 100)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 80)                57920     
__________

In [10]:
#compiling the regressor model

regressor.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

#training the the model

regressor.fit(x_train, y_train, epochs=150, batch_size=12)

Instructions for updating:
Use tf.cast instead.
Epoch 1/150
627/627 [==============================] - 7s 12ms/sample - loss: 0.6034 - acc: 0.7518
Epoch 2/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.5077 - acc: 0.7717
Epoch 3/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4967 - acc: 0.7739
Epoch 4/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4926 - acc: 0.7750
Epoch 5/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4892 - acc: 0.7756
Epoch 6/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4807 - acc: 0.7802
Epoch 7/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4742 - acc: 0.7847
Epoch 8/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4645 - acc: 0.7894
Epoch 9/150
627/627 [==============================] - 2s 3ms/sample - loss: 0.4564 - acc: 0.7942
Epoch 10/150
627/627 [==============================] - 2s 3ms/sample

## Preparing Test Data 

In [11]:
scaler.fit(test)
test = scaler.fit_transform(test)

x_test = []
y_test = []

for i in range(3, test.shape[0]):
    x_test.append(test[i-3:i])
    y_test.append(test[i])
    
# converting traning data into munpy array

x_test , y_test = np.array(x_test), np.array(y_test)

y_test = y_test[:,1:]

print(x_test.shape)
print(y_test.shape)

(97, 3, 30)
(97, 29)


C:\Users\FC\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\FC\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


## Test 

In [12]:
# predicting item
result = regressor.evaluate(x_test,y_test)

97/97 [==============================] - 1s 9ms/sample - loss: 0.2499 - acc: 0.9307


## Save model 

In [13]:
# regressor.save('todomodel.h5')